##### 第一种方法：利用正向传播，直接求解带参量子线路中的参数
具体的理论基础是对于|Φₒ> = 𝑼₅𝑼₄𝑼₃𝑼₃𝑼₂𝑼₁|Φ₀>，其对于𝑼₃的微分为𝑼₅𝑼₄∂𝑼₃𝑼₃𝑼₂𝑼₁|Φ₀>。因此只需要将𝑼₃门替换成对应的∂𝑼₃门，就可以求出grad。

In [1]:
using LinearAlgebra

# simple quantum system to implement quantum circuit
# quantum gates
# 注意！！！Ry(θ)在郭老师实现的版本中所有的θ都不加1/2π，这样的话，∂Ry(θ)就等于Ry(θ+1/2π)
# 但是如果像NC中这么定义的话，两个就不想等！！！
__CNOT = [1 0 0 0;0 1 0 0;0 0 0 1;0 0 1 0]
__Ry(θ) = [cos(θ/2) -sin(θ/2);sin(θ/2) cos(θ/2)]
__I = [1 0;0 1]
# matrix -> scalar's derivative == every element's derivative then transpose the matrix
__∂Ry(θ) = [-1/2sin(θ/2) -1/2cos(θ/2);1/2cos(θ/2) -1/2sin(θ/2)]

function CNOT(con, op, qubits=5)
    res = __CNOT
    for _ in 1:(con-1) res = kron(__I,res) end
    for _ in (op+1):qubits res = kron(res, __I) end
    return res
end

function __single_qubit(gate, op, qubits=5)
    res = gate
    for _ in 1:(op-1) res = kron(__I,res) end
    for _ in (op+1):qubits res = kron(res, __I) end
    return res
end

Ry(θ, op, qubits=5) = __single_qubit(__Ry(θ), op)
∂Ry(θ, op, qubits=5) = __single_qubit(__∂Ry(θ), op)

# quantum states
ϕ₀ = [1,0]
ϕ₁ = [0,1]

# ground state |00000> and target state |01101>
Φ₀ = kron(kron(kron(kron(ϕ₀, ϕ₀), ϕ₀), ϕ₀), ϕ₀)
Φᵩ = kron(kron(kron(kron(ϕ₀, ϕ₁), ϕ₁), ϕ₀), ϕ₁)

distance(Φ₁, Φ₂) = 1 - sum([Φ₁[x] * Φ₂[x] for x in 1:length(Φ₁)].^2)


# circuit
# --Ry₁---|---------------Ry₆---
# --Ry₂---⨁---|-----------Ry₇---
# --Ry₃-------⨁---|-------Ry₈---
# --Ry₄-----------⨁---|---Ry₉---
# --Ry₅---------------⨁---Ry₁₀--
function loss(params)
    temp = deepcopy(Φ₀)
    for x in 1:5 temp = Ry(params[x], x) * temp end # first layer's Ry
    for x in 1:4 temp = CNOT(x, x+1) * temp end # second layer's CNOT
    for x in 1:5 temp = Ry(params[x+5], x) * temp end # third layer's Ry
    return distance(temp, Φᵩ)
end

println(loss(randn(10).%2pi))

0.9998832015464804


In [2]:
#using Flux.Optimise
#using VQC

function __lossgrad(params, idx)
    
#     temp = deepcopy(Φ₀)
#     gates = [x != idx ? Ry(params[x], x) : ∂Ry(params[x], x) for x in 1:5]
#     gates = [gates;[x != idx ? Ry(params[x], x-5) : ∂Ry(params[x], x-5) for x in 6:10]]
    
    temp = deepcopy(Φ₀)
    temp_params = deepcopy(params)
    temp_params[idx] += 1/2pi
    gates = [[Ry(params[x], x) for x in 1:5];[Ry(params[x], x-5) for x in 6:10]]
    
    for x in 1:5 temp = gates[x] * temp end
    for x in 1:4 temp = CNOT(x, x+1) * temp end
    for x in 6:10 temp = gates[x] * temp end
    return distance(temp,Φᵩ)
end

function lossgrad(params)
    return [__lossgrad(params, idx) for idx in 1:10]
end

function numerical_lossgrad(params)
    ϵ = 0.001
    los = loss(params)
    dparams = []
    for x in 1:10
        params[x] += ϵ
        push!(dparams, (loss(params) - los) / ϵ)
        params[x] -= ϵ
    end
    return dparams
end

# init params
params = randn(10).%2pi

println(lossgrad(params))
println(numerical_lossgrad(params))


#=
# use ADAM as optimizer
opt = ADAM(0.01)
for i in 1:10000
    i % 1000 == 1 && println("loss value at epoch $i is $(loss(params)).")
    grad = lossgrad(params)
    Optimise.update!(opt, params, grad)
end=#

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/yanghanlin/.julia/packages/CUDAdrv/3EzC1/src/CUDAdrv.jl:69
┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1273


[0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551, 0.9995052588149551]
Any[0.00032644019087868514, 0.0026639101480263605, 0.0010461903973135733, -4.770810235754652e-5, -0.00041112482318528265, -0.0016567330261008095, -0.0024878979547615643, 0.00010765568969617334, -0.0018085963074954137, 0.00043787919945526]


In [3]:
final_params = params

for x in 1:5 Φ₀ = Ry(final_params[x], x) * Φ₀ end # first layer's Ry
for x in 1:4 Φ₀ = CNOT(x, x+1) * Φ₀ end # second layer's CNOT
for x in 1:5 Φ₀ = Ry(final_params[x+5], x) * Φ₀ end # third layer's Ry

println("the final_params will let Φ₀ reach Φᵩ with probability : ", projection(Φ₀, Φᵩ))

UndefVarError: UndefVarError: projection not defined